In [1]:
import numpy as np
import toml
from typing import List
from jinja2 import Template
import os
import moleculegraph
from scipy.constants import Avogadro

In [3]:
def get_name(name):
    if name[0] == "c":
        return name[1]
    else:
        return name[0]
    
def get_local_dipol( molecule: moleculegraph, force_field: dict[dict]) -> List[List]:
    """
    Function to get the local dipols of a molecule

    Args:
        molecule (moleculegraph): Moleculegraph object of the molecule under investigation.
        force_field (dict[dict]): Dictionary contains the force field types of the molecule under investigation.

    Returns:
        dipol_list (List[List]): List with sublists for each local dipol with the corresponding atom types in the dipol.
    """

    flag       = 0
    dipol_list = []
    dipol      = []

    # Search through all bonds and check if an atom is charged. If thats the case, start a local dipol list.
    # Check the neighboring atom (in the bond), and if its also charged, append it to the list and add up the local dipol charge. (if one atom is already in the local dipol list, skip the charge evaluation)
    # Once the charge is zero, a local dipol is identified and appended to the overall dipol list

    for bond_types in molecule.bond_list:

        for atom_type in bond_types:
            if atom_type in dipol: 
                continue

            elif force_field["atoms"][molecule.atom_names[atom_type]]["charge"] != 0 and flag == 0:
                dipol = [ atom_type ]
                chrg  = force_field["atoms"][molecule.atom_names[atom_type]]["charge"]
                flag  = 1
            
            elif force_field["atoms"][molecule.atom_names[atom_type]]["charge"] != 0 and flag == 1 :
                chrg += force_field["atoms"][molecule.atom_names[atom_type]]["charge"]
                dipol.append(atom_type)

                if np.round(chrg,3) == 0:
                    dipol_list.append(dipol)
                    flag = 0
            
    return dipol_list
        

In [12]:
class LAMMPS_input():

    def __init__(self, mol_str: List[str], ff_path: str, table_path: str=""):
        """
        Initilizing LAMMPS input class. Create playmol ff and save system independent force field parameters

        Args:
            mol_list (List[str]): List containing moleculegraph strings for the component(s). These will be transalted into moleculegraph objects
            ff_path (str): Path to toml file containing used force-field readable format by moleculegraph.
            table_path (str, optional): Provide a path to the table containing all tabled bonds (these include the nonbonded interactions evaluated in the charge group approach). 
                                        !! This is only necessary if the charge group approach is utilized. !!
        """

        # Save moleclue graphs of both components class wide
        self.mol_str  = mol_str
        self.mol_list = [ moleculegraph.molecule(mol) for mol in mol_str ]

        # Read in force field toml file
        with open(ff_path) as ff_toml_file:
            self.ff = toml.load(ff_toml_file)

        # If the charge group approach should be utilized, the force field as well as the molecule graph objects are altered to consider two local dipols within a molecule
        self.apply_charge_group_approach( table_path )


        ## Specify force field parameters for all interactions seperately (nonbonded, bonds, angles and torsions)

        # Get (unique) atom types and parameters #
        self.nonbonded = np.array([j for sub in [molecule.map_molecule( molecule.unique_atom_keys, self.ff["atoms"] ) for molecule in self.mol_list] for j in sub])
        
        # Get (unique) bond types and parameters #
        self.bonds     = [j for sub in [molecule.map_molecule( molecule.unique_bond_keys, self.ff["bonds"] ) for molecule in self.mol_list] for j in sub]
        
        # Get (unique) angle types and parameters #     
        self.angles    = [j for sub in [molecule.map_molecule( molecule.unique_angle_keys, self.ff["angles"] ) for molecule in self.mol_list] for j in sub]
        
        # Get (unique) torsion types and parameters #
        self.torsions  = [j for sub in [molecule.map_molecule( molecule.unique_torsion_keys, self.ff["torsions"] ) for molecule in self.mol_list] for j in sub]
        
        if not all( [ all([bool(entry) for entry in self.nonbonded]), all([bool(entry) for entry in self.bonds]), all([bool(entry) for entry in self.angles]), all([bool(entry) for entry in self.torsions]) ] ):
            txt = "nonbonded" if not all([bool(entry) for entry in self.nonbonded]) else "bonds" if not all([bool(entry) for entry in self.bonds]) else "angles" if not all([bool(entry) for entry in self.angles]) else "torsions"
            raise ValueError("Something went wrong during the force field typing for key: %s"%txt)
        
        # Get all atom types not only the unique one
        self.ff_all    = np.array([j for sub in [molecule.map_molecule( molecule.atom_names, self.ff["atoms"] ) for molecule in self.mol_list] for j in sub])


        ## Define general settings that are not system size dependent ##

        self.renderdict              = {}
        molecule1                    = self.mol_list[0]

        # Definitions for atoms in the system 
        # Always add 1, as the list starts at index 0. In case of mixture let types of molecule 2 start where types of molecule 1 ends
        add_atoms                    = [ 1, 1 + len(molecule1.unique_atom_indexes) ]

        self.number_of_atoms         = [ mol.atom_number for mol in self.mol_list ]
        self.atoms_running_number    = np.concatenate( [mol.unique_atom_inverse + add_atoms[i] for i,mol in enumerate(self.mol_list)], axis=0 )
        
        # Definitions for bonds in the system #
        # Always add 1, as the list starts at index 0. In case of mixture let types of molecule 2 start where types of molecule 1 ends
        add_bonds = [ 1, 1 + len(molecule1.unique_bond_keys) ]

        self.number_of_bonds         = [ len(mol.bond_keys) for mol in self.mol_list ]
        self.bonds_running_number    = np.concatenate( [mol.unique_bond_inverse + add_bonds[i] for i,mol in enumerate(self.mol_list)], axis=0 )
        self.bond_numbers            = np.concatenate( [mol.bond_list + 1 for mol in self.mol_list if mol.bond_list.size > 0], axis=0 )
        self.bond_names              = np.concatenate( [mol.bond_names for mol in self.mol_list if mol.bond_list.size > 0], axis=0 )
        self.bond_numbers_ges        = np.unique( self.bonds_running_number )

        self.renderdict["bond_styles"]        = list( np.unique( [ p["style"] for p in self.bonds ] ) )
        self.renderdict["bond_type_number"]   = len( self.bond_numbers_ges )

        # Definitions for angles in the system 
        # Always add 1, as the list starts at index 0. In case of mixture let types of molecule 2 start where types of molecule 1 ends
        add_angles = [ 1, 1+ len(molecule1.unique_angle_keys) ]

        self.number_of_angles        =[ len(mol.angle_keys) for mol in self.mol_list ]
        self.angles_running_number   = np.concatenate( [mol.unique_angle_inverse + add_angles[i] for i,mol in enumerate(self.mol_list)], axis=0 )
        self.angle_numbers           = np.concatenate( [mol.angle_list + 1 for mol in self.mol_list if mol.angle_list.size > 0], axis=0 )
        self.angle_names             = np.concatenate( [mol.angle_names for mol in self.mol_list if mol.angle_list.size > 0], axis=0 )
        self.angle_numbers_ges       = np.unique( self.angles_running_number )

        self.renderdict["angle_styles"]        = list( np.unique( [ p["style"] for p in self.angles] ) )
        self.renderdict["angle_type_number"]   = len(self.angle_numbers_ges)

        # Definitions for torsions in the system 
        # Always add 1, as the list starts at index 0. In case of mixture let types of molecule 2 start where types of molecule 1 ends
        add_torsions = [ 1, 1+ len(molecule1.unique_torsion_keys) ]
        
        self.number_of_torsions      = [ len(mol.torsion_keys) for mol in self.mol_list ]
        self.torsions_running_number = np.concatenate( [mol.unique_torsion_inverse + add_torsions[i] for i,mol in enumerate(self.mol_list)], axis=0 )
        self.torsion_numbers         = np.concatenate( [mol.torsion_list + 1 for mol in self.mol_list if mol.torsion_list.size > 0], axis=0 )
        self.torsion_names           = np.concatenate( [mol.torsion_names for mol in self.mol_list if mol.torsion_list.size > 0], axis=0 )
        self.torsion_numbers_ges     = np.unique( self.torsions_running_number )

        self.renderdict["torsion_styles"]      = list( np.unique( [ p["style"] for p in self.torsions ] ) )
        self.renderdict["torsion_type_number"] = len(self.torsion_numbers_ges)
        
        return

    def apply_charge_group_approach(self, table_path: str ):
        """
        Function that applies the charge group approach to the components. This function alters the bond entries in the moleculegraph objects, as well as the read in force field

        Args:
            table_path (str): _description_
        """

        self.table_path  = table_path
        self.dipol_lists = [ get_local_dipol(mol,self.ff) for mol in self.mol_list ]

        for dipol_list, mol in zip( self.dipol_lists, self.mol_list ):
            
            # If dipol list is empty or only contains one local dipol, skip !
            if len(dipol_list) < 2: continue

            # Use the dipol list, to create special bond list, as well as the corresponding moleculegraph representation of it
            special_bond_indexes = np.array( np.meshgrid( dipol_list[0], dipol_list[1] ) ).T.reshape(-1, 2)
            special_bond_names   = mol.atom_names[special_bond_indexes]
            special_bond_keys    = [ "[special]"+ moleculegraph.make_graph( moleculegraph.sort_force_fields(x) ) for x in special_bond_names ]

            # Delete unnecessary standard bonds. Checks which standard bonds also exist in the special bonds and therefore remove them
            idx             = np.array( [ i for i,entry in enumerate(mol.bond_list) if tuple(entry) not in set(map(tuple,special_bond_indexes)) ] )

            # Overwrite the bonding information of the moleculegraph object with the new bonds including the special bonds!
            mol.bond_list   = np.concatenate( [mol.bond_list[idx], special_bond_indexes], axis=0 )
            mol.bond_names  = np.concatenate( [mol.bond_names[idx], special_bond_names], axis=0 )
            mol.bond_keys   = np.concatenate( [mol.bond_keys[idx], special_bond_keys], axis=0 )

            mol.unique_bond_keys, mol.unique_bond_indexes, mol.unique_bond_inverse = moleculegraph.molecule_utils.unique_sort( mol.bond_keys, return_inverse=True )
            mol.unique_bond_names   = mol.bond_names[ mol.unique_bond_indexes ]
            mol.unique_bond_numbers = mol.bond_list[ mol.unique_bond_indexes ]    

            # Add the force field information for special bonds
            special_bond_keys_unique, uidx = np.unique(special_bond_keys, return_index=True)
            special_bond_names_unique      = special_bond_names[uidx]

            # Jinja2 template uses p[1] as first entry and p[0] as second
            for ( bkey, bname ) in zip(special_bond_keys_unique, special_bond_names_unique):
                self.ff["bonds"][bkey] = { "list": list(bname), "p":  [ bkey, table_path ], "style": "table", "type": -1}
        
        return 

    def prepare_playmol_input(self, playmol_template: str, playmol_ff_path: str):
        """
        Function that writes playmol force field using a jinja2 template.

        Args:
            playmol_template (str): Path to playmol template for system building.
            playmol_ff_path (str): Path were the new playmol force field file should be writen to.
        """

        # In case the charge group approach is used, playmol is build with a system disregarding this approach.
        # Instead unchangeed moleculegraph representations are used to produce playmol input.
        mol_list  =  [ moleculegraph.molecule(mol) for mol in self.mol_str ]

        # Get (unique) atom types and parameters #
        nonbonded = np.array([j for sub in [molecule.map_molecule( molecule.unique_atom_keys, self.copy_ff["atoms"] ) for molecule in mol_list] for j in sub])
        
        # Get (unique) bond types and parameters #
        bonds     = [j for sub in [molecule.map_molecule( molecule.unique_bond_keys, self.copy_ff["bonds"] ) for molecule in mol_list] for j in sub]
        
        # Get (unique) angle types and parameters #     
        angles    = [j for sub in [molecule.map_molecule( molecule.unique_angle_keys, self.copy_ff["angles"] ) for molecule in mol_list] for j in sub]
        
        # Get (unique) torsion types and parameters #
        torsions  = [j for sub in [molecule.map_molecule( molecule.unique_torsion_keys, self.copy_ff["torsions"] ) for molecule in mol_list] for j in sub]
     

        ## Prepare dictionary for jinja2 template to write force field input for Playmol ##

        renderdict              = {}
        renderdict["nonbonded"] = list( zip( [j for sub in [molecule.unique_atom_keys for molecule in self.mol_list] for j in sub], nonbonded ) )
        renderdict["bonds"]     = list( zip( [j for sub in [molecule.unique_bond_names for molecule in self.mol_list] for j in sub], bonds ) )
        renderdict["angles"]    = list( zip( [j for sub in [molecule.unique_angle_names for molecule in self.mol_list] for j in sub], angles ) )
        renderdict["torsions"]  = list( zip( [j for sub in [molecule.unique_torsion_names for molecule in self.mol_list] for j in sub], torsions ) )
        
        # Generate force field file for playmol using jinja2 template
        os.makedirs( os.path.dirname(playmol_ff_path), exist_ok=True )

        with open(playmol_template) as file_:
            template = Template(file_.read())
        
        rendered = template.render( rd=renderdict )

        with open(playmol_ff_path, "w") as fh:
            fh.write(rendered) 
    
        return
    
    def write_playmol_input(self, playmol_template: str, playmol_path: str, playmol_ff_path: str, xyz_paths: List[str], playmol_executeable: str="~/.local/bin/playmol"):
        """
        Function that generates input file for playmol to build the specified system, as well as execute playmol to build the system

        Args:
            playmol_template (str): Path to playmol input template.
            playmol_path (str): Path where the playmol .mol file is writen and executed.
            playmol_ff_path (str): Path to the playmol force field file.
            xyz_paths (List[str]): List with the path(s) to the xyz file(s) for each component.
            playmol_executeable (str, optional): Path to playmol executeable. Defaults to "~/.local/bin/playmol"
        """

        # In case the charge group approach is used, playmol is build with a system disregarding this approach.
        # Instead unchangeed moleculegraph representations are used to produce playmol input.
        mol_list      =  [ moleculegraph.molecule(mol) for mol in self.mol_str ]
                    
        moldict       = {}

        # Get running atom numbers
        add_atom      = [ 1, 1 + len(mol_list[0].unique_atom_keys)  ]
        atom_numbers  = list( np.concatenate( [ mol.unique_atom_indexes + add_atom[i] for i,mol in enumerate(mol_list) ] ) )
        
        # Get running bond numbers
        add_bonds     = [ 1, 1 + len(mol_list[0].unique_bond_keys) ]
        bond_numbers  = list(np.concatenate( [mol.bond_list + add_bonds[i] for i,mol in enumerate(mol_list) if mol.bond_list.size > 0], axis=0 ))

        # Get force field type 
        atom_names    = [j for sub in [molecule.atom_names for molecule in mol_list] for j in sub]

        # Generate names used for atoms in playmol != force field type --> to do so, add the running atom number to the name
        playmol_bond_names = list(np.concatenate( [mol.bond_names for mol in mol_list if mol.bond_list.size > 0], axis=0 ))

        # Playmol uses as atom input: atom_name force_field_type charge 
        moldict["atoms"]   = list(zip( atom_numbers, atom_names, [self.ff_all[i]["charge"] for i,_ in enumerate(atom_names)] ) )

        # Playmol uses as bond input: atom_name atom_name
        moldict["bonds"]   = list(zip( bond_numbers, playmol_bond_names))

        # Provide the number of molecules per component, as well as the starting atom of each molecule (e.g.: molecule1 = {C1, C2, C3}, molecule2 = {C4, C5, C6} --> Provide C1 and C4 )
        molecule_indices   = [ 0, mol_list[0].atom_number ] if len(mol_list) else [0]
        moldict["mol"]     = list( zip( self.nmol_list, [ str(moldict["atoms"][i][2])+str(moldict["atoms"][i][0]) for i in molecule_indices ] ) )

        # Add path to force field
        moldict["force_field"] = playmol_ff_path

        # Add path to xyz of one molecule of each component
        moldict["xyz"]         = xyz_paths

        # Add name of the final xyz file and log file
        moldict["final_xyz"]   = ".".join(os.path.basename(playmol_path).split(".")[:-1]) + ".xyz"
        moldict["final_log"]   = ".".join(os.path.basename(playmol_path).split(".")[:-1]) + ".log"


        ## Write playmol input file to build the system with specified number of molecules for each component ##

        with open(playmol_template) as file:
            template = Template(file.read())

        # Playmol template needs density in g/cm^3; rho given in kg/m^3 to convert in g/cm^3 divide by 1000
        rendered = template.render( rd   = moldict,
                                    rho  = str(self.density / 1000),
                                    seed = random.randint(1,1e6) )
        
        with open(playmol_path, "w") as fh:
            fh.write(rendered) 

        # Build system with playmol (in the specified folder)
        os.chdir( os.path.dirname(playmol_path) )
        log = os.system( "%s -i %s"%( playmol_executeable, os.path.basename(playmol_path) ) )
        print(log)
        print( "\nDONE: %s -i %s\n"%( playmol_executeable, os.path.basename(playmol_path) ) )

        return 
    
    def prepare_lammps_data(self, nmol_list: List, densitiy: float, decoupling: bool=False):
        """
        Function that prepares the LAMMPS data file at a given density for a given force field and system.

        Args:
            nmol_list (list): List containing the number of molecules per component
            densitiy (float): Mass density of the component/mixture at that state [kg/m^3]
            decoupling (bool,optional): If a decoupling input should be prepared. This will introduce new types for 
                                        the last molecule of component 1, to ensure that pair wise interactions can be 
                                        coupled / decoupled using a lambda parameter. Defaults to False.
                              
        """

        # Variables defined here are used class wide 
        self.nmol_list      = nmol_list
        self.density        = densitiy
        
        # Zip objects has to be refreshed for every system since its only possible to loop over them once
        self.renderdict["bond_paras"]    = zip(self.bond_numbers_ges, self.bonds)
        self.renderdict["angle_paras"]   = zip(self.angle_numbers_ges, self.angles)
        self.renderdict["torsion_paras"] = zip(self.torsion_numbers_ges, self.torsions)
        

        #### System specific settings ####

        ## Definitions for atoms in the system ##

        if ( nmol_list[0] > 1 and decoupling ):

            # In case decoupling simulations are performed:
            # Differentiate the last molecule of component 1 from every other 
            # Therefore create new types for last molecule and increase the type number of every atom in species 2 aswell

            self.add_type     = np.max( self.mol_list[0].unique_atom_indexes + 1 )

            a_list1           = self.mol_list[0].unique_atom_indexes + 1
            a_list1_add       = a_list1 + self.add_type
            a_list2           = self.mol_list[1].unique_atom_indexes + 1 + np.max(a_list1_add)

            self.pair_list    = list( np.concatenate( [ a_list1, a_list1_add, a_list2 ] ) )
            self.pair_ff      = list( self.nonbonded[a_list1-1], self.nonbonded[a_list1-1] ) + list( self.nonbonded[a_list2-1-self.add_type] )

        else:
            self.add_type     = 0

            add_atom          = [ 1, 1 + len(self.mol_list[0].unique_atom_keys)  ]

            self.pair_numbers = list( np.concatenate( [ mol.unique_atom_indexes + add_atom[i] for i,mol in enumerate(self.mol_list) ] ) )
            self.pair_ff      = list( self.nonbonded )


        self.renderdict["atom_paras"]       = list( zip(self.pair_numbers, self.pair_ff) )
        self.renderdict["atom_type_number"] = len(self.nonbonded) + self.add_type

        # Total atoms in system 
        self.total_number_of_atoms    = np.dot( self.number_of_atoms, nmol_list )

        # Total bonds in system 
        self.total_number_of_bonds    = np.dot( self.number_of_bonds, nmol_list )
        
        # Total angles in system 
        self.total_number_of_angles   = np.dot( self.number_of_angles, nmol_list) 
        
        # Total torsions in system 
        self.total_number_of_torsions = np.dot( self.number_of_torsions, nmol_list )


        ## Mass, mol, volume and box size of the system ##

        # Molar masses of each species [g/mol]
        #if not a["name"] == "H"
        Mol_masses = np.array( [ np.sum( [ a["mass"] for a in molecule.map_molecule( molecule.atom_names, self.ff["atoms"] ) ] ) for molecule in self.mol_list ] )

        # Account for mixture density --> in case of pure component this will not alter anything

        # mole fraction of mixture (== numberfraction)
        x = np.array( nmol_list ) / np.sum( nmol_list )

        # Average molar weight of mixture [g/mol]
        M_avg = np.dot( x, Mol_masses )

        # Total mole n = N/NA [mol] #
        n = np.sum( nmol_list ) / Avogadro

        # Total mass m = n*M [kg]
        mass = n * M_avg / 1000

        # Compute box volume V=m/rho and with it the box lenght L (in Angstrom) --> assuming orthogonal box
        # With mass (kg) and rho (kg/m^3 --> convert in g/A^3 necessary as lammps input)

        # Volume = mass / mass_density = mol / mol_density [A^3]
        volume = mass / self.density * 1e30

        boxlen = volume**(1/3) / 2

        box = [ -boxlen, boxlen ]

        self.renderdict["box_x"] = box
        self.renderdict["box_y"] = box
        self.renderdict["box_z"] = box
        
        return

def write_lammps_data(self, xyz_path: str, data_template: str, data_path: str):
        """
        Function that generates LAMMPS data file, containing bond, angle and torsion parameters, as well as all the coordinates etc.

        Args:
            xyz_path (str): Path to the xyz file for this system.
            data_template (str): Path to the jinja2 template for the LAMMPS data file.
            data_path (str): Path where the LAMMPS data file should be generated.
        """

        atom_count       = 0
        bond_count       = 0
        angle_count      = 0
        torsion_count    = 0
        mol_count        = 0
        mol_count1       = 0
        
        lmp_atom_list    = []
        lmp_bond_list    = []
        lmp_angle_list   = []
        lmp_torsion_list = []

        coordinates = moleculegraph.funcs.read_xyz(xyz_path)

        for m,mol in enumerate(self.mol_list):

            # All force field lists are created for both species --> only take part of the list for the specific component 

            # First component
            if m == 0:
                idx  = mol.atom_numbers
                idx1 = np.arange( len(self.mol_list[0].bond_keys) )
                idx2 = np.arange( len(self.mol_list[0].angle_keys) )
                idx3 = np.arange( len(self.mol_list[0].torsion_keys) )

            # Second component (add to the indices of the second component the number of atoms, bonds, angles and torsions of first component)
            elif m == 1:
                idx   = mol.atom_numbers + self.mol_list[0].atom_number
                idx1  = np.arange( len(self.mol_list[1].bond_keys) ) + len(self.mol_list[0].bond_keys)
                idx2  = np.arange( len(self.mol_list[1].angle_keys) ) + len(self.mol_list[0].angle_keys)
                idx3  = np.arange( len(self.mol_list[1].torsion_keys) ) + len(self.mol_list[0].torsion_keys)


            ## Now write LAMMPS input for every molecule of each component ##

            for mn in range(self.nmol_list[m]):
                
                # Define atoms

                for atomtype,ff_atom in zip( self.atoms_running_number[idx], self.ff_all[idx] ):
                    
                    atom_count +=1
                    
                    # These are coupling specific settings. If no coupling is wanted, add_type is 0 and no changes occur
                    # If last molecule of species 1: Add to every atomtype the number of types of species 1
                    if mn == self.nmol_list[0]-1 and m == 0:
                        atomtype += self.add_type
                    # If atoms of species 2, add to there atomtype to account for the extra types introduced (if N(species)==1 no special types are added (add_type =0))
                    elif m==1:
                        atomtype += self.add_type

                    # LAMMPS INPUT: total n° of atom in system, mol n° in System, atomtype, partial charges,coordinates
                    line = [ atom_count, mol_count+1, atomtype, ff_atom["charge"],*coordinates[atom_count-1]["xyz"], coordinates[atom_count-1]["atom"] ]

                    lmp_atom_list.append(line)


                # Define bonds

                for bondtype,bond,bond_name in zip( self.bonds_running_number[idx1], self.bond_numbers[idx1], self.bond_names[idx1] ):

                    bond_count += 1

                    # The bond counting for 2nd species needs to start after nmol1 * bonds_mol1 for right index of atoms
                    if m == 0:
                        dummy = bond + mol_count * mol.atom_number
                    elif m == 1:
                        dummy = bond + (mol_count-mol_count1) * mol.atom_number + mol_count1 * self.mol_list[0].atom_number

                    # LAMMPS INPUT: total n° of bonds in system, bond type, force field types in this bond
                    line = [ bond_count, bondtype, *dummy, " ".join(bond_name) ]

                    lmp_bond_list.append(line)

                # Define angles

                for angletype,angle,angle_name in zip (self.angles_running_number[idx2], self.angle_numbers[idx2], self.angle_names[idx2] ):

                    angle_count += 1

                    # The angles counting for 2nd species needs to start after nmol1 * angles_mol1 for right index of atoms
                    if m == 0:
                        dummy = angle + mol_count * mol.atom_number
                    elif m == 1:
                        dummy = angle + (mol_count-mol_count1) * mol.atom_number + mol_count1 * self.mol_list[0].atom_number

                    # LAMMPS INPUT: total n° of angles in system, angle type, force field types in this angle
                    line = [ angle_count, angletype, *dummy, " ".join(angle_name) ]

                    lmp_angle_list.append(line)

                # Define torsions

                for torsiontype,torsion,torsion_name in zip( self.torsions_running_number[idx3], self.torsion_numbers[idx3], self.torsion_names[idx3] ):

                    torsion_count += 1

                    # The torsion counting for 2nd species needs to start after nmol1 * torsion_mol1 for right index of atoms
                    if m == 0:
                        dummy = torsion + mol_count * mol.atom_number
                    elif m == 1:
                        dummy = torsion + (mol_count-mol_count1) * mol.atom_number + mol_count1 * self.mol_list[0].atom_number

                    dummy = np.flip( dummy ) # (?)
                    torsion = np.flip( torsion)  # (?)

                    # LAMMPS INPUT: total n° of torsions in system, torsion type, force field types in this torsion
                    line = [ torsion_count, torsiontype, *dummy, " ".join(torsion_name) ]

                    lmp_torsion_list.append(line)


                if m == 0: mol_count1 += 1

                mol_count += 1

                
            self.renderdict["atoms"]    = lmp_atom_list
            self.renderdict["bonds"]    = lmp_bond_list
            self.renderdict["angles"]   = lmp_angle_list
            self.renderdict["torsions"] = lmp_torsion_list

            self.renderdict["atom_number"]    = atom_count
            self.renderdict["bond_number"]    = bond_count
            self.renderdict["angle_number"]   = angle_count
            self.renderdict["torsion_number"] = torsion_count
            
        ## Write to jinja2 template file to create LAMMPS data file ##

        os.makedirs( os.path.dirname(data_path), exist_ok=True )

        with open(data_template) as file_:
            template = Template(file_.read())
            
        rendered = template.render( rd = self.renderdict )

        with open(data_path, "w") as fh:
            fh.write(rendered)

        return

In [6]:
ff = "./force-fields/forcefield_lammps.toml"

m1 = "[cH_alcohol][OH_alcohol][CH2_alcohol][CH2_alcohol][OH_alcohol][cH_alcohol]"
m2 = "[cH_alcohol][OH_alcohol][CH2_alcohol]"
m1 = moleculegraph.molecule(m1)
m2 = moleculegraph.molecule(m2)
m_list = [m1,m2]

a = LAMMPS_input( m_list, ff )